<h1 align=center><font size = 7><font color = red> Where's Wally Solver</font></h1>
  
<img src="Wally.png" style="width: 200px;float: center;"/>

This project uses Tensorflow Object Detection API to solve Where's Wally puzzle. The solver uses the Convolutional Neural Network EfficientDet D0 512x512 pre-trained model from  [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) and through Transfer Learning, it is retrained with a database of 32 Wally's face images. The model was trained for roughly 7 hours on Google Colab using a cloud GPU.

## Important:
Before running the model, you need to install Tensorflow and all the required libraries properly. [Gilbert Tanner has a great Youtube Tutorial](https://www.youtube.com/watch?v=cvyDYdI2nEI)

### Libraries 

In [1]:
# Make sure the required libraries are installed.
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim
!pip install pycocotools

import numpy as np
import tensorflow as tf
import pathlib
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import simpledialog
from tkinter import messagebox
import time


# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

  Using cached numpy-1.19.5-cp39-cp39-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.1
    Uninstalling numpy-1.20.1:
      Successfully uninstalled numpy-1.20.1


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


### Load Label Map

In [2]:
PATH_TO_LABELS = "object_detection/training/labelmap.pbtxt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Load Images

In [3]:
import pathlib
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

### Load Model

In [4]:
detection_model = tf.saved_model.load("object_detection/inference_graph/saved_model")

In [5]:
# Print the model signatures to make sure it is working properly
print(detection_model.signatures['serving_default'].output_dtypes)
print(detection_model.signatures['serving_default'].output_shapes)

{'detection_anchor_indices': tf.float32, 'detection_boxes': tf.float32, 'detection_classes': tf.float32, 'detection_multiclass_scores': tf.float32, 'detection_scores': tf.float32, 'num_detections': tf.float32, 'raw_detection_boxes': tf.float32, 'raw_detection_scores': tf.float32}
{'detection_anchor_indices': TensorShape([1, 100]), 'detection_boxes': TensorShape([1, 100, 4]), 'detection_classes': TensorShape([1, 100]), 'detection_multiclass_scores': TensorShape([1, 100, 1]), 'detection_scores': TensorShape([1, 100]), 'num_detections': TensorShape([1]), 'raw_detection_boxes': TensorShape([1, 49104, 4]), 'raw_detection_scores': TensorShape([1, 49104, 1])}


### Inference Functions - Tensorflow Colab

In [6]:
def run_inference_for_single_image(model, image):
    
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [8]:
def show_inference(model, image_path):
  
  Msgflag = 0
  img = 0
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      groundtruth_box_visualization_color='blue',
      use_normalized_coordinates=True,
      line_thickness=5)
  
  if (output_dict['detection_scores'][0])>=0.5:
    img = Image.fromarray(image_np)
  else:
    Msgflag = 1

  return Msgflag, img


### Detect Wally

In [ ]:
# Image selection
root = tk.Tk()
root.withdraw()
answer = simpledialog.askstring("Select Image", "Select an image between 0 and 17",
                                parent=root)
# Showing image
if answer != None:
    path = str(TEST_IMAGE_PATHS[int(answer)])
    img = Image.open(path)  
    img.show()
    
# Allowed time to find Wally in Sec
    time.sleep(10)
    root = tk.Tk()
    root.withdraw()
    MsgBox = messagebox.askyesno ('Hey','Did you find Wally?',icon = 'question', parent=root)

# Detect Wally
    if MsgBox == False:

        MsgBox, Img = show_inference(detection_model, TEST_IMAGE_PATHS[int(answer)])
        if MsgBox == 0:
            root = tk.Tk()
            root.withdraw()
            Ok = messagebox.showinfo("Hey","Wally Found!")
            if Ok == "ok":
                Img.show()
        if MsgBox == 1:
            root = tk.Tk()
            root.withdraw()
            messagebox.showinfo ('Hey','Sorry, Wally not found')